<a href="https://colab.research.google.com/github/huysuy05/LLM_Evals_On_Imbalanced_Datatset/blob/main/eval_with_hgf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install torch numpy pandas transformers bitsandbytes huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 11.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

print(torch.mps.is_available())

True


In [20]:
import pandas as pd

label_map = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech"
}
imbalanced_data = pd.read_parquet("Data/ag_news_train_imbalanced.parquet")
balanced_data = pd.read_parquet("Data/ag_news_train_balanced.parquet")
testing_set = pd.read_parquet("Data/ag_news_test_small.parquet")

balanced_data["label_word"] = balanced_data["label"].map(label_map)
imbalanced_data["label_word"] = imbalanced_data["label"].map(label_map)
testing_set["label_word"] = testing_set["label"].map(label_map)

# Shuffle the dataset
imbalanced_data = imbalanced_data.sample(frac=1).reset_index(drop=True)
balanced_data = balanced_data.sample(frac=1).reset_index(drop=True)
testing_set = testing_set.sample(frac=1).reset_index(drop=True)

imbalanced_data.shape


(2600, 3)

In [14]:
!pip install hf_xet

In [21]:
import pandas as pd


def classify(text, labels, model):
  """
    Use a Pipeline from Transformers for zero shot classification

    Args:
      text (str): The text data per row
      labels (List[int]): A list of label
      model (str): Model name to load
  """
  from transformers import pipeline

  classifier = pipeline("zero-shot-classification", model=model)
  result = classifier(text, labels)
  pred_label = result["labels"][0]
  return pred_label

In [22]:
def evaluate(data, labels, model):
  total_samples = data.shape[0]
  correct_pred = 0

  print(f"=== Starting evaluation ===")

  for idx, row in data.iterrows():
    text = row["text"]
    correct_label = row["label_word"].lower()
    pred_label = classify(text, labels, model)
    pred_label = pred_label.lower()

    if pred_label:
      if pred_label == correct_label:
        correct_pred += 1
        print(f"Article {idx}: Correctly classified as '{correct_label}'")
      else:
          print(f"Article {idx}: Incorrectly classified. True: '{correct_label}', Predicted: '{pred_label}'")
  acc = (correct_pred / total_samples) * 100
  return acc


In [ ]:
# Test the model
labels = ["World", "Sports", "Business", "Sci/Tech"]

model = "facebook/bart-large-mnli"

print("Evaluating LLM on Balanced Data...")
balanced_accuracy = evaluate(testing_set, labels, model)
print(f"\nAccuracy on Balanced Data: {balanced_accuracy:.2f}%")

print("\n\nEvaluating LLM on Imbalanced Data...")
imbalanced_accuracy = evaluate(imbalanced_data, labels, model)
print(f"\nAccuracy on Imbalanced Data: {imbalanced_accuracy:.2f}%")



Evaluating LLM on Imbalanced Data...
=== Starting evaluation ===


Device set to use mps:0


Article 0: Correctly classified as 'world'


Device set to use mps:0


Article 1: Incorrectly classified. True: 'world', Predicted: 'business'


...

Article 2597: Correctly classified as 'world'


Device set to use mps:0


Article 2598: Correctly classified as 'world'


Device set to use mps:0


Article 2599: Incorrectly classified. True: 'world', Predicted: 'sci/tech'

Accuracy on Imbalanced Data: 50.19%
